<a href="https://colab.research.google.com/github/sistelca/NOT-MNIST/blob/master/3_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Learning
##Assignment 3
Anteriormente, en 2_fullyconnected.ipynb, entrenó una regresión logística y un modelo de red neuronal.

El objetivo de esta tarea es explorar técnicas de regularización.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Primero recarga los datos que generamos en 1_notmnist.ipynb

In [0]:
# corre solo para descomprimir la primera vez

from zipfile import ZipFile

filename = "/content/gdrive/My Drive/curso_udacity/notMNIST.zip"

	
with ZipFile(filename) as zipObj:
  # Extract all the contents of zip file in current directory
  zipObj.extractall("/content/gdrive/My Drive/curso_udacity/")

In [0]:
pickle_file = '/content/gdrive/My Drive/curso_udacity//notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)
  
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0]) 


# Cambie el formato para adaptarlo a los modelos que vamos a entrenar:
# datos como una matriz plana, Las etiquetas como float 1 codifican en caliente.
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)


image_size = 28
num_labels = 10
batch_size = 128
image_size = 28


Training set (50000, 28, 28) (50000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


#Problem 1
Introduzca y ajuste la regularización de L2 para los modelos de redes neuronales y logísticas. Recuerde que L2 equivale a agregar una penalización en la norma de los pesos a la pérdida. En TensorFlow, puede calcular la pérdida de L2 para un tensor t utilizando nn.l2_loss (t). La cantidad correcta de regularización debería mejorar su validación / precisión de prueba.

In [0]:
# regularizacion L2 para logística

graph = tf.Graph()
with graph.as_default() as defa:

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)+0.01*tf.nn.l2_loss(weights))
      
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("** Test presicion Global: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.579704
Minibatch accuracy: 13.3%
Validation accuracy: 18.0%
Minibatch loss at step 500: 0.849907
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Minibatch loss at step 1000: 0.921997
Minibatch accuracy: 75.8%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.860147
Minibatch accuracy: 77.3%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 0.666245
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 0.780673
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 0.714072
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
** Test presicion Global: 88.1%


In [0]:
# regularizacion para la red neuronal
hidden_nodes = 1024 # neuronas en la primera capa oculta
lamda_regula = .01
tf_dropout_rate = 0.5
  
with graph.as_default() as gradef:
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    # Capa 1
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    # Capa 2
    #w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes//2]))
    #b_2 = tf.Variable(tf.zeros([hidden_nodes//2]))
    
    # Capa 3 de salida
    w_3 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_3 = tf.Variable(tf.zeros([num_labels]))
    
    def NN(x, tf_dropout_rate = tf_dropout_rate): # neurona
        """
            x: matriz
               su forma debe ser (m, 784)

            regresa la activacion de la capa de salida
            matriz de (m, 10)
        """

        # Capa oculta 1
        z_1 = tf.matmul(x,w_1) + b_1 # lineal
        a_1 = tf.nn.relu(z_1)	# activacion
        # apply DropOut to hidden layer
        drop_out = tf.nn.dropout(a_1, tf_dropout_rate)  # DROP-OUT here
        
        # Capa oculta 2
#        z_2 = tf.matmul(drop_out1, w_2) + b_2
#        a_2 = tf.nn.relu(z_2)
#        drop_out = tf.nn.dropout(a_2, tf_dropout_rate)  # DROP-OUT here

        # Capa 2 salida
        z_3 = tf.matmul(drop_out,w_3) + b_3

        return z_3
    
    # Training computation.
    
    y_ = NN(tf_train_dataset, tf_dropout_rate)  # dropout
    
    # funcion de costo
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=y_)+
    lamda_regula*tf.nn.l2_loss(w_1)+
    lamda_regula*tf.nn.l2_loss(w_2))
    
    # Optimizador.
    optim = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(y_)
    valid_prediction = tf.nn.softmax(NN(tf_valid_dataset, 1.0))
    test_prediction = tf.nn.softmax(NN(tf_test_dataset, 1.0))
    

  

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optim, loss, y_], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test presicion Global: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5648.840820
Minibatch accuracy: 7.0%
Validation accuracy: 26.1%
Minibatch loss at step 500: 40.619511
Minibatch accuracy: 72.7%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 1.382952
Minibatch accuracy: 70.3%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 0.904320
Minibatch accuracy: 79.7%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 0.728635
Minibatch accuracy: 85.2%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 0.779911
Minibatch accuracy: 80.5%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.702906
Minibatch accuracy: 82.8%
Validation accuracy: 82.3%
Test presicion Global: 88.8%


#Problem 2 y 3
- Vamos a demostrar un caso extremo de sobreeentrenamiento. Limite sus datos de entrenamiento a unos pocos lotes. ¿Qué es lo que pasa?

- Introduce Dropout en la capa oculta de la red neuronal. Recuerda: La deserción sólo debe introducirse durante la capacitación, no durante la evaluación, de lo contrario, los resultados de la evaluación también serían estocásticos. TensorFlow proporciona nn.dropout() para ello, pero debes asegurarte de que sólo se inserta durante el entrenamiento.



In [0]:
# trabajaremos solo el caso de redes neuronales
# se observa que ante una dramatica disminucion del numero de muestras
# la regularizacion es necesaria para prevenir sobreentrenamiento
# mas esto no mejora la presicion global del modelo;
# en general cambiar los parametros de regularizacion no inside en 
# la precision resultante

redneu(.01, 0.5)
ejecute(train_dataset[0:1000, :], train_labels[0:1000, :])

Initialized
Minibatch loss at step 0: 16109.240234
Minibatch accuracy: 13.3%
Validation accuracy: 31.4%
Minibatch loss at step 500: 1.206977
Minibatch accuracy: 80.5%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 1.005678
Minibatch accuracy: 88.3%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 1.111877
Minibatch accuracy: 83.6%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 0.925378
Minibatch accuracy: 89.8%
Validation accuracy: 78.8%
Minibatch loss at step 2500: 1.103470
Minibatch accuracy: 82.8%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 1.169951
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Test presicion Global: 84.2%


#Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.

    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


In [0]:
# al aumentar el numero de capas se redujo significativamente
# la presicion del modelo
# probamos ahora con learnid decay
hidden_nodes = 1024 # neuronas en la primera capa oculta
lamda_regula = .01
tf_dropout_rate = 0.5
  
with graph.as_default() as gradef:
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    global_step = tf.Variable(0)
    
    # Capa 1
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    # Capa 2
    #w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes//2]))
    #b_2 = tf.Variable(tf.zeros([hidden_nodes//2]))
    
    # Capa 3 de salida
    w_3 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_3 = tf.Variable(tf.zeros([num_labels]))
    
    
    def NN(x, tf_dropout_rate = tf_dropout_rate): # neurona
        """
            x: matriz
               su forma debe ser (m, 784)

            regresa la activacion de la capa de salida
            matriz de (m, 10)
        """

        # Capa oculta 1
        z_1 = tf.matmul(x,w_1) + b_1 # lineal
        a_1 = tf.nn.relu(z_1)	# activacion
        # apply DropOut to hidden layer
        drop_out = tf.nn.dropout(a_1, tf_dropout_rate)  # DROP-OUT here
        
        # Capa oculta 2
#        z_2 = tf.matmul(drop_out, w_2) + b_2
#        a_2 = tf.nn.relu(z_2)
#        drop_out = tf.nn.dropout(a_2, tf_dropout_rate)  # DROP-OUT here

        # Capa 2 salida
        z_3 = tf.matmul(drop_out,w_3) + b_3

        return z_3
    
    # Training computation.
    
    y_ = NN(tf_train_dataset, tf_dropout_rate)  # dropout
    
    learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.6)
    
    # funcion de costo
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=y_)+
    lamda_regula*tf.nn.l2_loss(w_1)+
    lamda_regula*tf.nn.l2_loss(w_2))
    
    # Optimizador.
    optim = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(y_)
    valid_prediction = tf.nn.softmax(NN(tf_valid_dataset, 1.0))
    test_prediction = tf.nn.softmax(NN(tf_test_dataset, 1.0))

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optim, loss, y_], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test presicion Global: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5612.298828
Minibatch accuracy: 8.6%
Validation accuracy: 28.9%
Minibatch loss at step 500: 1902.742065
Minibatch accuracy: 77.3%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 701.119751
Minibatch accuracy: 78.9%
Validation accuracy: 84.1%
Minibatch loss at step 1500: 257.295105
Minibatch accuracy: 78.1%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 94.139992
Minibatch accuracy: 85.9%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 34.898041
Minibatch accuracy: 85.2%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 13.171215
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Test presicion Global: 92.4%
